In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import anndata as an
import scipy
from scipy.sparse import csr_matrix
import sklearn
import gget
import scanpy.external as sce
import math

sc.settings.verbosity = 3  

import utils as ut
import plotting as plt2

In [2]:
fpath = "/scratch/indikar_root/indikar1/shared_data/sc_HSC/scanpy/merged_scfib.h5ad"
adata = sc.read_h5ad(fpath)
adata.obs['UMAP 1'] = adata.obsm['X_umap'][:, 0]
adata.obs['UMAP 2'] = adata.obsm['X_umap'][:, 1]
adata.obs['cluster_name'] = adata.obs['new_cluster'].apply(lambda x : f"F{int(x) + 1}")
sc.logging.print_memory_usage()
adata

Memory usage: current 8.08 GB, difference +8.08 GB


AnnData object with n_obs × n_vars = 16130 × 13579
    obs: 'cluster_str', 'dataset', 'pred_phase', 'leiden', 'n_genes', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'new_cluster', 'UMAP 1', 'UMAP 2', 'cluster_name'
    var: 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'cluster_str_colors', 'dataset_colors', 'hvg', 'log1p', 'neighbors', 'new_cluster', 'new_cluster_colors', 'pca', 'pred_phase_colors', 'umap'
    obsm: 'X_pca', 'X_scanorama', 'X_umap'
    varm: 'PCs'
    layers: 'combat', 'counts', 'log_norm', 'magic', 'norm'
    obsp: 'connectivities', 'distances'

In [3]:
fpaths = [
    "../resources/PanglaoDB_Augmented_2021.txt",
    "../resources/Tabula_Sapiens.txt",
]

marker_columns = []

for fpath in fpaths:    
    features = ut.load_pathway(fpath)

    # extract columns
    hsc_columns = [x for x in features.columns if "hemato" in x.lower()]
    fib_columns = [x for x in features.columns if "fibroblast" in x.lower()]
    fib_columns = [x for x in fib_columns if not "myofibr" in x.lower()]
    
    columns = list(set(hsc_columns + fib_columns))

    for col in columns:
        gene_list = list(features[features[col].astype(bool)].index)
        print(col, len(gene_list))
        col_name = col.lower().replace("-", " ")
        col_name = col_name.replace(" ", "_") + "_marker"
        marker_columns.append(col_name)
        adata.var[col_name] = adata.var.index.isin(gene_list)
    
adata.var['is_marker'] = adata.var[marker_columns].any(axis=1)
print()
adata

Hematopoietic Stem Cells 178
Fibroblasts 232
Large Intestine-fibroblast 100
Salivary Gland-fibroblast 100
Thymus-fibroblast 100
Fat-fibroblast 100
Uterus-fibroblast 100
Blood-hematopoietic Stem Cell 100
Tongue-fibroblast 100
Trachea-fibroblast 100
Small Intestine-fibroblast 100
Lung-alveolar Fibroblast 100
Liver-fibroblast 100
Pancreas-fibroblast 100
Spleen-hematopoietic Stem Cell 100
Heart-fibroblast Of Cardiac Tissue 100
Lung-fibroblast 100
Bladder-fibroblast 100
Eye-fibroblast 100
Lymph Node-hematopoietic Stem Cell 100
Vasculature-fibroblast 100
Mammary-fibroblast Of Breast 100
Prostate-fibroblast 100
Bone Marrow-hematopoietic Stem Cell 100



AnnData object with n_obs × n_vars = 16130 × 13579
    obs: 'cluster_str', 'dataset', 'pred_phase', 'leiden', 'n_genes', 'n_genes_by_counts', 'total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'new_cluster', 'UMAP 1', 'UMAP 2', 'cluster_name'
    var: 'n_cells', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'hematopoietic_stem_cells_marker', 'fibroblasts_marker', 'large_intestine_fibroblast_marker', 'salivary_gland_fibroblast_marker', 'thymus_fibroblast_marker', 'fat_fibroblast_marker', 'uterus_fibroblast_marker', 'blood_hematopoietic_stem_cell_marker', 'tongue_fibroblast_marker', 'trachea_fibroblast_marker', 'small_in

In [4]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [12]:
qdata = sc.get.aggregate(
    adata, 
    by='cluster_name', 
    func='mean', 
    axis='obs',
    layer='log_norm',
)

qdata.to_df(layer='mean').T.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9
A1BG,0.079666,0.239177,0.298476,0.103064,0.092488,0.045310,0.027176,0.021441,0.228143
A2M,0.002609,0.005922,0.003844,0.009739,0.000000,0.013155,0.004468,0.000000,0.008152
A2ML1,0.323652,1.382022,0.606320,0.633825,0.262116,0.963210,0.442686,0.321410,0.375305
A3GALT2,0.126543,0.106807,0.087357,0.097866,0.107519,0.171692,0.135239,0.089970,0.088248
A4GNT,0.004571,0.000000,0.000000,0.003157,0.011020,0.004657,0.000000,0.000000,0.000000


# Counts

In [ ]:
"""Count of cells non-zero"""

qdata = sc.get.aggregate(
    adata, 
    by='dataset', 
    func='count_nonzero', 
    axis='obs',
    layer='counts',
)

query = [
    'CD34',
    'GATA2',
    'GFI1B',
    'FOS',
    'STAT5A',
    'REL',
    'CD164',
]

counts = qdata[:, query].to_df(layer='count_nonzero')
counts = counts.divide(adata.obs['dataset'].value_counts(), axis=0) * 100
counts = counts.T
counts = counts.reset_index(drop=False, names='gene_name')
counts = pd.melt(counts, id_vars='gene_name')

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = len(query) / 1.5, 3

sns.barplot(
    data=counts,
    x='gene_name',
    y='value',
    hue='variable',
    ec='k',
    palette=["#FFA500", "#008080"],
    width=0.65,
)

plt.gca().tick_params(axis='x', rotation=90)
plt.ylabel('% non-zero')
plt.xlabel('')
sns.despine()

sns.move_legend(
    plt.gca(),
    title='',
    loc='best',
)

In [ ]:
fpath = "../config/gene_annotations/scenic_transcription_factors.csv"
scenic = pd.read_csv(fpath)
print(f"{scenic.shape=}")
tf_list = scenic['gene_name']
scenic.head()

In [ ]:
counts = qdata.to_df(layer='count_nonzero').T
counts = counts.divide(adata.obs['dataset'].value_counts(), axis=1) * 100

counts['diff'] = counts['iHSC'] - counts['scFib']
counts = counts.sort_values(by='diff', ascending=False)
counts = counts.reset_index(drop=False, names='gene_name')
counts = counts[counts['gene_name'].isin(tf_list)]
counts.head(15)

In [ ]:
# break

In [ ]:
n_genes = 15
database = 'ontology'
query = list(counts['gene_name'].head(n_genes))
edf = gget.enrichr(query, database=database)

pd.set_option('display.max_colwidth', 100)
edf[['path_name', 'adj_p_val', 'overlapping_genes']].head(15)


In [ ]:
"""mean expression"""

qdata = sc.get.aggregate(
    adata, 
    by='dataset', 
    func='mean', 
    axis='obs',
    layer='log_norm',
)

query = [
    'CD34',
    'GATA2',
    'GFI1B',
    'FOS',
    'STAT5A',
    'REL',
    'CD164',
]

means = qdata[:, query].to_df(layer='mean').T

means = means.reset_index(drop=False, names='gene_name')
means = pd.melt(means, id_vars='gene_name')

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = len(query) / 1.5, 3

sns.barplot(
    data=means,
    x='gene_name',
    y='value',
    hue='variable',
    ec='k',
    palette=["#FFA500", "#008080"],
    width=0.65,
)

plt.gca().tick_params(axis='x', rotation=90)
plt.ylabel('mean expression')
plt.xlabel('')
sns.despine()

sns.move_legend(
    plt.gca(),
    title='',
    loc='best',
)

In [ ]:
means = qdata.to_df(layer='mean').T
means['diff'] = means['iHSC'] - means['scFib']
means = means.sort_values(by='diff', ascending=False)
means.head(15)

In [ ]:
n_genes = 50
database = 'ontology'
query = list(means.head(n_genes).index)
edf = gget.enrichr(query, database=database)

pd.set_option('display.max_colwidth', 100)
edf[['path_name', 'adj_p_val', 'overlapping_genes']].head(15)

# Cluster aggregations

In [ ]:
qdata = sc.get.aggregate(
    adata, 
    by='cluster_name', 
    func='count_nonzero', 
    axis='obs',
    layer='counts',
)

query = [
    'CD34',
    'GATA2',
    'GFI1B',
    'FOS',
    'STAT5A',
    'REL',
    'CD164',
]

counts = qdata[:, query].to_df(layer='count_nonzero')
counts = counts.divide(adata.obs['cluster_name'].value_counts(), axis=0) * 100
counts = counts.T
counts = counts.reset_index(drop=False, names='gene_name')
counts = pd.melt(counts, id_vars='gene_name')

plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = len(query) / 1, 3

sns.barplot(
    data=counts,
    x='gene_name',
    y='value',
    hue='variable',
    ec='k',
    width=0.65,
)

plt.gca().tick_params(axis='x', rotation=90)
plt.ylabel('% non-zero')
plt.xlabel('')
sns.despine()

sns.move_legend(
    plt.gca(),
    title='',
    loc='center right',
    bbox_to_anchor=(1.5, 0.5)
)

In [ ]:
counts = qdata.to_df(layer='count_nonzero').T
counts['HSC_like'] = counts[['F4', 'F7']].sum(axis=1)
counts['FB_like'] = counts[['F1', 'F2','F3', 'F5', 'F6']].sum(axis=1)
counts['diff'] = counts['HSC_like'] - counts['FB_like']

counts = counts.sort_values(by='diff', ascending=False)
counts.head(15)

In [ ]:
n_genes = 25
database = 'ontology'
query = list(counts.head(n_genes).index)
edf = gget.enrichr(query, database=database)

pd.set_option('display.max_colwidth', 100)
edf[['path_name', 'adj_p_val', 'overlapping_genes']].head(15)

# Marker Genes

In [ ]:
"""mean expression"""

qdata = sc.get.aggregate(
    adata, 
    by='cluster_name', 
    func='mean', 
    axis='obs',
    layer='log_norm',
)

qdata

In [ ]:
result = []

que = 'hemato'

for cell_type in marker_columns:
    if not que in cell_type.lower():
        continue

    cell_name = cell_type.replace("_marker", "")
    cell_name = " ".join(cell_name.split("_"))
    marker_genes = list(qdata.var[qdata.var[cell_type]].index)
    df = qdata[:, marker_genes].to_df(layer='mean').mean(axis=1)
    row = df.to_dict()
    row['cell_type'] = cell_name
    result.append(row)
    
result = pd.DataFrame(result)
print(f"{result.shape=}")
result = result.set_index('cell_type')
result = result.drop(columns=['F8', 'F9'])
result.head()

m, n = result.shape
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = n/1.75, n

g = sns.clustermap(
    result,
    figsize=(n / 1, m / 2),
    square=True,
    lw=1,
    z_score=1,
    linecolor='k',
)


plt.ylabel("")
plt.xlabel("")
plt.show()

In [ ]:
result = []

que = 'fibro'

for cell_type in marker_columns:
    if not que in cell_type.lower():
        continue
        
    marker_genes = list(qdata.var[qdata.var[cell_type]].index)
    df = qdata[:, marker_genes].to_df(layer='mean').mean(axis=1)
    row = df.to_dict()
    row['cell_type'] = cell_type
    result.append(row)
    
result = pd.DataFrame(result)
print(f"{result.shape=}")
result = result.set_index('cell_type')
result = result.drop(columns=['F8', 'F9'])
result.head()

m, n = result.shape
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = n/1.75, n

g = sns.clustermap(
    result,
    figsize=(n / 1, m / 2),
    square=True,
    lw=1,
    z_score=1,
    linecolor='k',
)

plt.ylabel("")
plt.xlabel("")
plt.show()